# Gene Regulatory Network Tutorial

In this tutorial we run through GRN graph construction in Graphein. This works by providing a `gene_list` containing gene names. The interactions between these genes are retrieved from two possible sources:
* TRRUST
* RegNetwork

This is controlled by providing an edge construction function (`add_trrust_edges` and `add_regnetwork_edges`) respectively.

In [ ]:
import networkx as nx
from functools import partial
from typing import List
import matplotlib.pyplot as plt

from graphein.grn.edges import add_regnetwork_edges, add_trrust_edges

## Config Objects
Global parameters are stored in Config objects. We have a global `GRNGraphConfig` that contains a `TRRUSTConfig` and a `RegNetworkConfig` for parameters relating to each of the two sources.

In [ ]:
# Initialise Config Object
from graphein.grn.config import GRNGraphConfig
config = GRNGraphConfig()
print(f"GRN Config: \n{config} \n")
print(f"TRRUST Config: \n{config.trrust_config} \n")
print(f"RegNetwork Config: \n{config.regnetwork_config}")

## Gene List
This is the list of genes that we wish to construct a GRN graph for

In [ ]:
gene_list: List[str] = ["AATF", "MYC", "USF1", "SP1", "TP53", "DUSP1"]

## Edge Annotation Functions
These are functions that determine labelling of edges with additional metadata. Here we define one which assigs:
    * `"+"` to upregulatory interactions
    * `"-"` to downregulatory interactions
    * `"?"` to interactons of unknown directionality
    
Edge Annotation functions take in two nodes (`u`, `v`) and the edge data (`d`)

In [ ]:
# This function defines the labelling of regulatory directions
def edge_ann_fn(u, v, d):
    if "+" in d["regtype"]:
        d["regtype"] = "+"
    elif "-" in d["regtype"]:
        d["regtype"] = "-"
    elif "?" in d["regtype"]:
        d["regtype"] = "?"

## Graph Construction

In [ ]:
from graphein.grn.graphs import compute_grn_graph
from graphein.grn.features.node_features import add_sequence_to_nodes

g = compute_grn_graph(
    gene_list=gene_list,
    edge_construction_funcs=[
        partial(add_trrust_edges, trrust_filtering_funcs=config.trrust_config.filtering_functions),
        partial(add_regnetwork_edges, regnetwork_filtering_funcs=config.regnetwork_config.filtering_functions),
    ],
    node_annotation_funcs=[add_sequence_to_nodes],#, molecular_weight],
    edge_annotation_funcs=[edge_ann_fn],
)

In [ ]:
for n,d in g.nodes(data=True):
    print(n, d)

## Plotting the Graph
We now wish to visualise the graph. We can do this with the built in visualisation in NetworkX

In [ ]:
# Here we define a colouring scheme for the graph
edge_colors = [
    "r"
    if g[u][v]["kind"] == {"trrust"}
    else "b"
    if g[u][v]["kind"] == {"regnetwork"}
    else "y"
    for u, v in g.edges()
]

print(nx.info(g))

pos = nx.spring_layout(g)
nx.draw(g, pos=pos, with_labels=True, edge_color=edge_colors)
edge_labels = {(u, v): g[u][v]["regtype"] for u, v in g.edges}
nx.draw_networkx_edge_labels(g, pos=pos, edge_labels=edge_labels)
plt.show()

In [ ]:
print(g.edges(data=True))